In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('../input/demand-forecasting-kernels-only/train.csv', parse_dates=['date'])
test = pd.read_csv('../input/demand-forecasting-kernels-only/test.csv', parse_dates=['date'])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import datetime
from sklearn.model_selection import train_test_split, TimeSeriesSplit
import xgboost as xgb
from sklearn.metrics import mean_absolute_error

In [ ]:
def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

In [ ]:
test.columns


In [ ]:
train.columns


In [ ]:
print(train.shape,test.shape)


In [ ]:
test.tail()


In [ ]:
print('store'	, ' values: ', train['store'].unique())
print('item'	, ' values: ', train['item'].unique())
print('store'	, ' values: ', test['store'].unique())
print('item'	, ' values: ', test['item'].unique())

In [ ]:
plt.subplot(3, 1, 1)
plt.plot(train.sales[:365*4])
plt.subplot(3, 1, 2)
plt.plot(train.sales[:365])

plt.subplot(3, 1, 3)
plt.plot(train.sales[:31])
plt.show()

In [ ]:
rolling_mean = train.sales.rolling(window=7).mean()
print(rolling_mean.shape)
print(rolling_mean[:31])

In [ ]:
rolling_mean = train.sales.rolling(window=7).mean()
plt.subplot(3, 1, 1)
plt.plot(rolling_mean[:365*4])
plt.subplot(3, 1, 2)
plt.plot(rolling_mean[:365])

plt.subplot(3, 1, 3)
plt.plot(rolling_mean[:31])
plt.show()

In [ ]:
rolling_mean2 = train.sales.rolling(window=30).mean()
plt.subplot(2, 1, 1)
plt.plot(rolling_mean2[:365*4])
plt.subplot(2, 1, 2)
plt.plot(rolling_mean2[:365])
plt.show()

In [ ]:

data_combine = pd.concat([train,test])

print("size of data_combine",data_combine.shape)

In [ ]:
data_combine['date'] = pd.to_datetime(data_combine['date'],infer_datetime_format=True)


data_combine['month'] = data_combine['date'].dt.month
data_combine['weekday'] = data_combine['date'].dt.dayofweek
data_combine['year'] = data_combine['date'].dt.year
# df['date'].dt.
data_combine['week_of_year']  = data_combine.date.dt.weekofyear

In [ ]:
data_combine['date_order'] = (data_combine['date'] - datetime.datetime(2013, 1, 1)).dt.days


In [ ]:
data_combine.head(100)


In [ ]:
data_combine['sale_moving_average_7days']=data_combine.groupby(["item","store"])['sales'].transform(lambda x: x.rolling(window=7,min_periods=1).mean())
data_combine['sale_moving_average_7days_shifted-90']=data_combine.groupby(["item","store"])['sale_moving_average_7days'].transform(lambda x:x.shift(90))

In [ ]:
data_combine['store_item_shifted-90'] = data_combine.groupby(["item","store"])['sales'].transform(lambda x:x.shift(90))
#data_list2=data_combine.groupby(["item","store"])['sales'].transform(lambda x:x.shift(90))
#print(data_list2)

In [ ]:
data_combine.head(n=40)


In [ ]:
data_combine['store_item_shifted-10'] = data_combine.groupby(["item","store"])['sales'].transform(lambda x:x.shift(10))


In [ ]:
data_combine.head(n=40)


In [ ]:
data_combine.tail(n=40)


In [ ]:

col = [i for i in data_combine.columns if i not in ['date','id','sale_moving_average_7days']]
y_target = train.sales

In [ ]:
train_new = data_combine.loc[~data_combine.sales.isna()]
print("new train",train_new.shape)
test_new = data_combine.loc[data_combine.sales.isna()]
print("new test",test_new.shape)

In [ ]:
train_new = (train_new[col]).dropna()


In [ ]:
print(train_new.shape)


In [ ]:
y_target = train_new.sales
col = [i for i in data_combine.columns if i not in ['date','id','sales','sale_moving_average_7days']]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( train_new[col] ,train_new.sales, test_size=0.15, random_state=42)


In [ ]:
y_train.head()


In [ ]:
model_sets=[]

for max_depth in range(4,17,3):
  xgb_model = xgb.XGBRegressor(max_depth=max_depth ,min_child_weight=1)
#xgb_model.fit(X_train,y_train,eval_metric='mae')
  xgb_model.fit(X_train,y_train.values,eval_metric=smape)
  model_sets.append(xgb_model)
  
  y_train_pred_xgb=xgb_model.predict(X_train)
  y_test_pred_xgb=xgb_model.predict(X_test)
  print('smape error: max_depth=', max_depth ,',train:' , smape(y_train.values,y_train_pred_xgb),'test:',smape(y_test.values,y_test_pred_xgb))
  print('MSE train:' , mean_absolute_error(np.log1p(y_train),np.log1p(y_train_pred_xgb)),'test:',mean_absolute_error(np.log1p(y_test),np.log1p(y_test_pred_xgb)))

In [ ]:
#xgb_model = xgb.XGBRegressor(max_depth=6,min_child_weight=1)
#xgb_model.fit(X_train,y_train,eval_metric='mae')
model_sets[2]
model_sets[2].fit(train_new[col],y_target,eval_metric=smape)

y_train_pred_xgb=model_sets[2].predict(X_train)
y_test_pred_xgb=model_sets[2].predict(X_test)
print('smape error: max_depth=', max_depth ,',train:' , smape(y_train.values,y_train_pred_xgb),'test:',smape(y_test.values,y_test_pred_xgb))
print('MSE train:' , mean_absolute_error(np.log1p(y_train),np.log1p(y_train_pred_xgb)),'test:',mean_absolute_error(np.log1p(y_test),np.log1p(y_test_pred_xgb)))

In [ ]:

# Choose the model which provides less smape
y_submission=np.rint(model_sets[2].predict(test_new[col]))
#y_submission=xgb_model.predict(test_new[col])

In [ ]:
final = pd.DataFrame(list(zip(test_new['id'], y_submission)), 
               columns =['id', 'sales'])

In [ ]:
final.id = final.id.astype(int)
final.sales = final.sales.astype(int)

In [ ]:
final.head(n=20)


In [ ]:
final.to_csv("submission.csv",sep=',', index=False)


In [ ]:
y_submission=xgb_model.predict(test[col])